In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')
import warnings; warnings.simplefilter('ignore')

/home/klyshko/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/klyshko/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('lyrics_titles_AutoPump.csv')

In [3]:
all_lyrics = open('all_lyrics.txt', 'w')

In [4]:
data.head()

,Unnamed: 0,title,lyrics
0,0,1 Choppa 2 Choppa*,"[Intro]\r\nAyy, ayy\r\n\r\n[Chorus]\r\nOne cho..."
1,0,30's,"[Verse 1: Lil Pump]\r\nI'ma hit a stain, I'ma ..."
2,0,70 Nigga,\r\n Lyrics for this song...
3,0,Aight,"[Intro: Lil Pump]\r\nYuh, ouu, ouu, ouu, ouu, ..."
4,0,At the Door,"[Intro]\r\nOoh, Big Head on the beat\r\nLil Pu..."


In [5]:
for row in data.itertuples():
    text = row.lyrics
    all_lyrics.write(text)

In [6]:
all_lyrics.close()

In [7]:
all_lyrics = open('all_lyrics.txt', 'r')
for line in all_lyrics:
    if line.startswith('  '):
        print(line)

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro: Lil Pump]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro: Lil Pump]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Verse]          

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro: Rick Ross]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

   

In [8]:
lines = []
all_lyrics = open('all_lyrics.txt', 'r')
for line in all_lyrics:
    if line.startswith(' '):
        line = line.lstrip()
    line = line.lower()
    if line.startswith('[chorus'):
        lines.append('[chorus] \n')
    elif line.startswith('[verse'):
        lines.append('[verse] \n')
    elif line.startswith('[hook'):
        lines.append('[hook] \n')
    elif line.startswith('[intro'):
        lines.append('[intro] \n')
    elif line.startswith('[outro'):
        lines.append('[outro] \n')
    elif line.startswith('[bridge'):
        lines.append('[bridge] \n')
    elif line.startswith('[interlude'):
        lines.append('[interlude] \n')
    elif line.startswith('lyrics for this'):
        lines.append('\n')
    else:
        lines.append(line)
all_lyrics.close()

In [9]:
all_lyrics_cleaned = open('all_lyrics_cleaned.txt', 'w')
for line in lines:
    all_lyrics_cleaned.write(line)
all_lyrics_cleaned.close() 

In [10]:
# Read in the entire file.
f = open('all_lyrics_cleaned.txt')
corpus0 = f.read()
f.close()

# Separate the punctuation from the words, so that words with
# punctuation do not get counted as distinct from words without
# punctuation.  Same for new line characters.

corpus0 = corpus0.replace(',', ' ,')
corpus0 = corpus0.replace('(', ' ( ')
corpus0 = corpus0.replace(')', ' ) ')
corpus0 = corpus0.replace('[', ' [ ')
corpus0 = corpus0.replace(']', ' ] ')
corpus0 = corpus0.replace('.', ' . ')
corpus0 = corpus0.replace(';', ' ; ')
corpus0 = corpus0.replace(':', ' : ')
corpus0 = corpus0.replace('!', ' ! ')
corpus0 = corpus0.replace('?', ' ? ')
corpus0 = corpus0.replace('*', ' * ')
corpus0 = corpus0.replace("’", '\'')
corpus0 = corpus0.replace("\'\'", ' " ')
corpus0 = corpus0.replace('"', ' " ')
corpus0 = corpus0.replace('\r\n', ' \r\n ')

# Separate the dashes from any words they're attached to.
corpus0 = corpus0.replace('-', ' - ')
corpus0 = corpus0.replace('\n', ' \n ')

# Convert the text to lower case.
corpus0 = corpus0.lower()

# Split the words by spaces; only take the first 500000 words.
# This number was chosen based on memory limits and training-time
# limitations.
corpus1 = corpus0.split(' ')

while (corpus1.count('') > 0): 
    corpus1.remove('')
    
print('Length of corpus is now ', len(corpus1))

Length of corpus is now  46686


In [31]:
# Preprocessing is done.  Now get the unique words, and encode them.
words = sorted(list(set(corpus1)))
num_words = len(words)
encoding = {w: i for i, w in enumerate(words)}
decoding = {i: w for i, w in enumerate(words)}

print('We have', num_words, 'different words.')
corpus = corpus1

We have 2476 different words.


In [57]:
# Chop up the data into x and y, slice into roughly num_chars
# overlapping 'sentences' of length sentence_length.  Encode the
# characters.
sentence_length = 10
x_data = []
y_data = []
for i in range(0, len(corpus) - sentence_length):
    sentence = corpus[i: i + sentence_length]
    next_word = corpus[i + sentence_length]
    x_data.append([encoding[word] for word in sentence])
    y_data.append(encoding[next_word])

# good word: phronesis
num_sentences = len(x_data)
print('We have', len(x_data), 'sentences.')

# Create the variables to hold the data as it will be used.
x = np.zeros((num_sentences, sentence_length, num_words), dtype = np.bool)
y = np.zeros((num_sentences, num_words), dtype = np.bool)

# Populate the sentences.
print('Encoding data.')
for i, sentence in enumerate(x_data):
    for t, encoded_word in enumerate(sentence):
        x[i, t, encoded_word] = 1
    y[i, y_data[i]] = 1

We have 46676 sentences.
Encoding data.


In [33]:
# The processing of the data takes a fair amount of time.  Save
# the data so we don't have to do this again.  We do this in a
# numpy file since the data is large and the shelve can't handle
# it.

print('Saving processed data.')
np.save('x.npy', x)
np.save('y.npy', y)

Saving processed data.


In [34]:
import keras.models as km
import keras.layers as kl

In [35]:
print('Building network.')
model = km.Sequential()
model.add(kl.LSTM(64, input_shape = (sentence_length, num_words)))
model.add(kl.Dense(num_words, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd',
              metrics = ['accuracy'])

Building network.


In [36]:
# Fit!  Begin elevator music...
print('Beginning fit.')
fit = model.fit(x, y, epochs = 200, batch_size = 128)

# Save the model so that we can use it as a starting point.
model.save('model.h5')

Beginning fit.
Epoch 1/200
46676/46676 [==============================] - 20s 429us/step - loss: 7.7659 - acc: 0.1000
Epoch 2/200
46676/46676 [==============================] - 20s 426us/step - loss: 7.6643 - acc: 0.1016
Epoch 3/200
46676/46676 [==============================] - 20s 424us/step - loss: 7.5387 - acc: 0.1016
Epoch 4/200
46676/46676 [==============================] - 20s 424us/step - loss: 7.2959 - acc: 0.1016
Epoch 5/200
46676/46676 [==============================] - 20s 424us/step - loss: 6.6887 - acc: 0.1016
Epoch 6/200
46676/46676 [==============================] - 20s 424us/step - loss: 6.2307 - acc: 0.1016
Epoch 7/200
46676/46676 [==============================] - 20s 424us/step - loss: 5.9877 - acc: 0.1016
Epoch 8/200
46676/46676 [==============================] - 20s 424us/step - loss: 5.8042 - acc: 0.1016
Epoch 9/200
46676/46676 [==============================] - 20s 424us/step - loss: 5.6766 - acc: 0.1016
Epoch 10/200
46676/46676 [==============================] 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



46676/46676 [==============================] - 20s 426us/step - loss: 5.1479 - acc: 0.1251
Epoch 134/200
46676/46676 [==============================] - 20s 426us/step - loss: 5.1463 - acc: 0.1262
Epoch 135/200
46676/46676 [==============================] - 20s 435us/step - loss: 5.1447 - acc: 0.1279
Epoch 136/200
46676/46676 [==============================] - 20s 426us/step - loss: 5.1430 - acc: 0.1261
Epoch 137/200
46676/46676 [==============================] - 20s 426us/step - loss: 5.1416 - acc: 0.1277
Epoch 138/200
46676/46676 [==============================] - 20s 426us/step - loss: 5.1401 - acc: 0.1289
Epoch 139/200
46676/46676 [==============================] - 20s 426us/step - loss: 5.1386 - acc: 0.1296
Epoch 140/200
46676/46676 [==============================] - 20s 427us/step - loss: 5.1371 - acc: 0.1291
Epoch 141/200
46676/46676 [==============================] - 20s 427us/step - loss: 5.1354 - acc: 0.1293
Epoch 142/200
46676/46676 [==============================] - 20s 427u

In [37]:
fit = model.fit(x, y, epochs = 200, batch_size = 128)

# Save the model so that we can use it as a starting point.
model.save('model.h5')

Epoch 1/200
46676/46676 [==============================] - 20s 427us/step - loss: 4.9661 - acc: 0.1712
Epoch 2/200
46676/46676 [==============================] - 20s 427us/step - loss: 4.9606 - acc: 0.1737
Epoch 3/200
46676/46676 [==============================] - 20s 427us/step - loss: 4.9556 - acc: 0.1740
Epoch 4/200
46676/46676 [==============================] - 20s 427us/step - loss: 4.9492 - acc: 0.1766
Epoch 5/200
46676/46676 [==============================] - 20s 426us/step - loss: 4.9438 - acc: 0.1781
Epoch 6/200
46676/46676 [==============================] - 20s 426us/step - loss: 4.9389 - acc: 0.1803
Epoch 7/200
46676/46676 [==============================] - 20s 428us/step - loss: 4.9346 - acc: 0.1809
Epoch 8/200
46676/46676 [==============================] - 20s 426us/step - loss: 4.9291 - acc: 0.1836
Epoch 9/200
46676/46676 [==============================] - 20s 427us/step - loss: 4.9248 - acc: 0.1844
Epoch 10/200
46676/46676 [==============================] - 20s 427us/ste

46676/46676 [==============================] - 20s 429us/step - loss: 4.4771 - acc: 0.2600
Epoch 158/200
46676/46676 [==============================] - 20s 428us/step - loss: 4.4742 - acc: 0.2606
Epoch 159/200
46676/46676 [==============================] - 20s 428us/step - loss: 4.4704 - acc: 0.2616
Epoch 160/200
46676/46676 [==============================] - 20s 427us/step - loss: 4.4674 - acc: 0.2633
Epoch 161/200
46676/46676 [==============================] - 20s 428us/step - loss: 4.4637 - acc: 0.2628
Epoch 162/200
46676/46676 [==============================] - 20s 427us/step - loss: 4.4605 - acc: 0.2628
Epoch 163/200
46676/46676 [==============================] - 20s 428us/step - loss: 4.4570 - acc: 0.2649
Epoch 164/200
46676/46676 [==============================] - 20s 428us/step - loss: 4.4553 - acc: 0.2640
Epoch 165/200
46676/46676 [==============================] - 20s 428us/step - loss: 4.4511 - acc: 0.2645
Epoch 166/200
46676/46676 [==============================] - 20s 428u

In [ ]:
fit = model.fit(x, y, epochs = 200, batch_size = 128)

# Save the model so that we can use it as a starting point.
model.save('model.h5')

Epoch 1/200
46676/46676 [==============================] - 22s 463us/step - loss: 0.9019 - acc: 0.8333
Epoch 2/200
46676/46676 [==============================] - 21s 445us/step - loss: 0.9049 - acc: 0.8324
Epoch 3/200
46676/46676 [==============================] - 21s 445us/step - loss: 0.8999 - acc: 0.8323
Epoch 4/200
46676/46676 [==============================] - 21s 447us/step - loss: 0.9229 - acc: 0.8274
Epoch 5/200
46676/46676 [==============================] - 21s 446us/step - loss: 0.9222 - acc: 0.8273
Epoch 6/200
46676/46676 [==============================] - 21s 442us/step - loss: 0.9278 - acc: 0.8261
Epoch 7/200
46676/46676 [==============================] - 21s 448us/step - loss: 0.9044 - acc: 0.8319
Epoch 8/200
46676/46676 [==============================] - 21s 442us/step - loss: 0.9163 - acc: 0.8288
Epoch 9/200
46676/46676 [==============================] - 21s 443us/step - loss: 0.9115 - acc: 0.8300
Epoch 10/200
46676/46676 [==============================] - 21s 443us/ste

In [55]:
import numpy as np

import keras.models as km
import shelve
import random

modelfile = 'model.h5'

#######################################################################

# Get the model.
print('Reading model file.')
model = km.load_model(modelfile)

# Get the meta-data.

sentence_length = 10
num_words = 2476


# Randomly choose 50 words from the dictionary of words as our
# starting sentence.
seed = []
for i in range(sentence_length):
    seed.append(decoding[random.randint(0, num_words - 1)])


# Encode the seed sentence.
x = np.zeros((1, sentence_length, num_words), dtype = np.bool)
for i, w in enumerate(seed):
    x[0, i, encoding[w]] = 1

text = ''

# Run the seed sentence through the model.  Add the output to the
# generated text.  Take the output and append it to the seed sentence
# and remove the first word from the seed sentence.  Then repeat until
# you've generated as many words as you like.
for i in range(150):

    # Get the most-probably next word.
    pred = np.argmax(model.predict(x, verbose = 0))

    # Add it to the generated text.
    text += decoding[pred] + ' '

    # Encode the next word.
    next_word = np.zeros((1, 1, num_words), dtype = np.bool)
    next_word[0, 0, pred] = 1

    # Concatenate the next word to the seed sentence, but leave off
    # the first element so that the length stays the same.
    x = np.concatenate((x[:, 1:, :], next_word), axis = 1)

    
# Print out the generated text.
print("Generated lyrics: \n")
print(text)

Reading model file.
Generated lyrics: 

( skrr ) 
 yeah ) , my lean , i don't broke the ( okay ) 
 i pump the school , do you a couple ) 
 all you you do it just me me like me . yeah 
 yeah [ chorus : : : ] 
 lil pump 
 [ chorus ] 
 ouu , yuh 
 yuh , yuh , yeah 
 boppin' [ chorus ] 
 ayy , no bitch , no know go flex ? ) 
 bitch i think go cars , pack 
 louis and lil pump , no all , i got my wrist ( ooh ) for that four on the wrist 
 and ride designer pack like the damn 
 
 [ chorus ] 
 fuck so bitches in the trap 
 
 [ verse ] 
 lil pump told that lil' pump , i got up in 
